In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark GBTRegressor example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/18 03:22:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/18 03:22:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/02/18 03:22:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
df = spark.read.format('com.databricks.spark.csv').\
                               options(header='true', \
                               inferschema='true').\
                               load("data/Advertising.csv",header=True);

df.show(5,True)
df.printSchema()

+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
+-----+-----+---------+-----+
only showing top 5 rows

root
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)



In [3]:
df.describe().show()

24/02/18 03:23:02 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+------------------+------------------+------------------+
|summary|               TV|             Radio|         Newspaper|             Sales|
+-------+-----------------+------------------+------------------+------------------+
|  count|              200|               200|               200|               200|
|   mean|         147.0425|23.264000000000024|30.553999999999995|14.022500000000003|
| stddev|85.85423631490805|14.846809176168728| 21.77862083852283| 5.217456565710477|
|    min|              0.7|               0.0|               0.3|               1.6|
|    max|            296.4|              49.6|             114.0|              27.0|
+-------+-----------------+------------------+------------------+------------------+



Convert the data to dense vector (features and label)

In [4]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

transformed=df.rdd.map(lambda r: [Vectors.dense(r[:-1]),r[-1]]).toDF(['features','label'])
transformed.show(5)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[230.1,37.8,69.2]| 22.1|
| [44.5,39.3,45.1]| 10.4|
| [17.2,45.9,69.3]|  9.3|
|[151.5,41.3,58.5]| 18.5|
|[180.8,10.8,58.4]| 12.9|
+-----------------+-----+
only showing top 5 rows



Deal with the Categorical variables

In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

featureIndexer = VectorIndexer(inputCol="features", \
                               outputCol="indexedFeatures",\
                               maxCategories=4).fit(transformed)

data = featureIndexer.transform(transformed)
data.show(5,True)

+-----------------+-----+-----------------+
|         features|label|  indexedFeatures|
+-----------------+-----+-----------------+
|[230.1,37.8,69.2]| 22.1|[230.1,37.8,69.2]|
| [44.5,39.3,45.1]| 10.4| [44.5,39.3,45.1]|
| [17.2,45.9,69.3]|  9.3| [17.2,45.9,69.3]|
|[151.5,41.3,58.5]| 18.5|[151.5,41.3,58.5]|
|[180.8,10.8,58.4]| 12.9|[180.8,10.8,58.4]|
+-----------------+-----+-----------------+
only showing top 5 rows



Split the data into training and test sets (40% held out for testing)

In [6]:
# Split the data into training and test sets (40% held out for testing)
(trainingData, testData) = data.randomSplit([0.6, 0.4])

trainingData.show(5)
testData.show(5)

+---------------+-----+---------------+
|       features|label|indexedFeatures|
+---------------+-----+---------------+
| [0.7,39.6,8.7]|  1.6| [0.7,39.6,8.7]|
| [5.4,29.9,9.4]|  5.3| [5.4,29.9,9.4]|
|[7.3,28.1,41.4]|  5.5|[7.3,28.1,41.4]|
|[7.8,38.9,50.6]|  6.6|[7.8,38.9,50.6]|
| [8.4,27.2,2.1]|  5.7| [8.4,27.2,2.1]|
+---------------+-----+---------------+
only showing top 5 rows

+----------------+-----+----------------+
|        features|label| indexedFeatures|
+----------------+-----+----------------+
|  [4.1,11.6,5.7]|  3.2|  [4.1,11.6,5.7]|
| [13.1,0.4,25.6]|  5.3| [13.1,0.4,25.6]|
|[13.2,15.9,49.6]|  5.6|[13.2,15.9,49.6]|
|[17.2,45.9,69.3]|  9.3|[17.2,45.9,69.3]|
|[17.9,37.6,21.6]|  8.0|[17.9,37.6,21.6]|
+----------------+-----+----------------+
only showing top 5 rows



Fit RandomForest Regression Model with GBTRegressor

In [7]:
# Import LinearRegression class
from pyspark.ml.regression import GBTRegressor

# Define LinearRegression algorithm
gbt = GBTRegressor() #numTrees=2, maxDepth=2, seed=42

If you decide to use the indexedFeatures features, you need to add the parameter featuresCol="indexedFeatures".

Pipeline Architecture

In [8]:
# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, gbt])
model = pipeline.fit(trainingData)

Make predictions

In [9]:
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("features","label", "prediction").show(5)

24/02/18 03:23:59 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


+----------------+-----+-----------------+
|        features|label|       prediction|
+----------------+-----+-----------------+
|  [4.1,11.6,5.7]|  3.2|5.132674876007165|
| [13.1,0.4,25.6]|  5.3|5.145373662243048|
|[13.2,15.9,49.6]|  5.6|6.992983868472848|
|[17.2,45.9,69.3]|  9.3| 9.43868845548319|
|[17.9,37.6,21.6]|  8.0|7.749033576102263|
+----------------+-----+-----------------+
only showing top 5 rows



Evaluation

In [10]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 1.18684


In [11]:
evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="r2")
print("R Squared (R2) on test data = %g" % evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.939112


Feature importances

In [12]:
model.stages[-1].featureImportances

SparseVector(3, {0: 0.6246, 1: 0.3355, 2: 0.0399})

In [13]:
model.stages[-1].trees

[DecisionTreeRegressionModel: uid=dtr_977a4d87fbeb, depth=5, numNodes=63, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_51bd1b2ea591, depth=5, numNodes=47, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_0c0ac4993626, depth=5, numNodes=47, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_6f7f28305192, depth=5, numNodes=45, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_873ef539feb1, depth=5, numNodes=41, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_80918ff0f98d, depth=5, numNodes=45, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_29496130d791, depth=5, numNodes=45, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_69616c038dec, depth=5, numNodes=47, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_d209c959b8d4, depth=5, numNodes=45, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_df17ab1fb35e, depth=5, numNodes=55, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_5d3bed0f6948, depth=5, numNodes=43, numFeatures=3,
 DecisionTreeRegressi